In [1]:
import torch
import torch.nn as nn
import os
import requests
import tiktoken
import pandas as pd
import math

In [2]:
token_length= 16
d_model = 64
batch_size = 4

In [3]:
with open('dataset.txt', 'r') as f:
    text = f.read()



In [4]:
encoding = tiktoken.get_encoding('cl100k_base')
tokenized_text = encoding.encode(text)
tokenized_text = torch.tensor(tokenized_text, dtype=torch.long)

In [5]:
train_index = int(len(tokenized_text) * 0.9)
train_data = tokenized_text[:train_index]
validation_data = tokenized_text[train_index:]

In [6]:
data = train_data
indexs = torch.randint(low=0, high=len(data) - token_length, size=(batch_size,))
# 4 * 16 tensor
x_batch = torch.stack([data[idx: idx + token_length] for idx in indexs])
y_batch = torch.stack([data[idx + 1: idx + token_length + 1] for idx in indexs])

In [7]:
# pd.DataFrame(x_batch[0].numpy())
encoding.decode(x_batch[0].numpy())

' or over the phone\nHandling objections in written communication or over the phone is an'

In [8]:
# get max index of token
max_token_value = tokenized_text.max().item()
max_token_value

100069

In [9]:
# construct a 100070 * 16 matrix, get input embedding
input_embedding_table = nn.Embedding(max_token_value + 1, d_model)
input_embedding_table.weight.data

tensor([[-0.1733, -0.3731,  0.2028,  ..., -0.0237, -0.3536,  1.3827],
        [ 0.4144, -0.0151,  1.2830,  ...,  1.2278,  0.5334,  0.0971],
        [-0.5918, -0.2436, -0.4565,  ..., -0.3854,  0.5225,  0.9887],
        ...,
        [ 0.2023, -1.8109,  0.9871,  ..., -1.7215, -0.7932, -0.3979],
        [-1.3804, -0.1381, -1.2865,  ...,  1.7921,  0.1391,  0.5029],
        [ 0.5249, -1.2645, -0.4546,  ..., -0.4729,  1.0280, -1.0916]])

In [10]:
x_batch_embedding = input_embedding_table(x_batch)
y_batch_embedding = input_embedding_table(y_batch)
x_batch_embedding.shape

torch.Size([4, 16, 64])

In [14]:
# get postional encoding
# first 16 * 16 matrix
postion_encoding_table = torch.zeros(token_length, d_model)
# postion 16 * 1
postion = torch.arange(0, token_length, dtype=torch.float).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
# [:, 0::2], ':' = selecting all lines, '0::2' = starting from 0, step by step 2
postion_encoding_table[:, 0::2] = torch.sin(postion * div_term)
postion_encoding_table[:, 1::2] = torch.sin(postion * div_term)
postion_encoding_table = postion_encoding_table.unsqueeze(0).expand(batch_size, -1, -1)

postion_encoding_table.shape

torch.Size([4, 16, 64])

In [16]:
x = x_batch_embedding + postion_encoding_table
y = y_batch_embedding + postion_encoding_table
x.shape

torch.Size([4, 16, 64])